# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [26]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

In [55]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        '''ToDo: fix this function'''
        return 1.0 * self.w / self.C

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        if 1 - answer * self.f(x) > 0:
            return -1.0
        else:
            return 0.

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** k

            # w update            
            '''ToDo: add w update with regularization'''
            self.w -= (step * y * x * self.der_loss(x, y) + self.der_reg())
            
            # w_0 update
            '''ToDo: add w_0 update'''
            self.w0 -= step * y * self.der_loss(x, y)

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [56]:
model = MySVM()
model.fit(X_train, y_train)
print(model.w, model.w0)

[-0.15150201  0.73784895  0.53013729  0.82508488  0.46781213  0.57546931
 -0.68727899 -0.12405884  0.01799554 -0.75831006 -0.34147456 -0.2552856
  0.55307224 -0.90056568 -0.5996077  -0.02057609  0.12817311  1.7560642
  0.85404949  0.11445516] -0.8289694298107237


In [57]:
predictions = model.predict(X_test)

In [58]:
print(predictions)

[0 1 0 ... 0 1 1]


In [59]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ... 1 0 1] 2000 991


In [60]:
print(len(predictions), sum(predictions))

2000 1143


In [61]:
print(sum(predictions == y_test) / float(len(y_test)))

0.61


## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

In [62]:
from sklearn.svm import LinearSVC

In [72]:
modelSklearn= LinearSVC(C=10000, max_iter=100000)

In [73]:
modelSklearn.fit(X_train, y_train)
predictionsSklearn = modelSklearn.predict(X_test)
print(sum(predictionsSklearn == y_test) / float(len(y_test)))

0.614


/home/abdrakhimov/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
